In [1]:
import os
import pickle
import numpy as np
import pandas as pd
# from emvert_functional_code import emvert_ml
from scipy.sparse import csgraph
import csv
import shutil

import argparse
import os
import pickle
import random
import sys
import tempfile
import time

import gc
import matplotlib.cm
import networkx as nx
import numpy as np
import scipy.sparse as spsprs
import torch
import torch.autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os
import csv
import numpy as np
import scipy.io as sio

from sklearn.linear_model import RidgeClassifier
from sklearn.feature_selection import RFE
from nilearn import connectome
from collections import Counter
from nilearn import datasets
import matplotlib.pyplot as plt

In [2]:
pipeline = 'cpac'

# Input data variables
root_folder = '/data/zfzhu/zs/MMGL/miccai_ABIDE/data'
data_folder = os.path.join(root_folder, 'ABIDE_pcp/cpac/filt_noglobal')
phenotype = os.path.join(root_folder, 'ABIDE_pcp/Phenotypic_V1_0b_preprocessed1.csv')
#phenotype = os.path.join(root_folder, 'TADPOLE/processed_standard_data.csv')
def fetch_filenames(subject_IDs, file_type):

    """
        subject_list : list of short subject IDs in string format
        file_type    : must be one of the available file types
    returns:
        filenames    : list of filetypes (same length as subject_list)
    """

    import glob

    # Specify file mappings for the possible file types
    filemapping = {'func_preproc': '_func_preproc.nii.gz',
                   'rois_ho': '_rois_ho.1D'}

    # The list to be filled
    filenames = []

    # Fill list with requested file paths
    for i in range(len(subject_IDs)):
        
        os.chdir(os.path.join(data_folder, subject_IDs[i]))  # os.path.join(data_folder, subject_IDs[i]))
        try:
            filenames.append(glob.glob('*' + subject_IDs[i] + filemapping[file_type])[0])
        except IndexError:
            # Return N/A if subject ID is not found
            filenames.append('N/A')

    return filenames


# Get timeseries arrays for list of subjects
def get_timeseries(subject_list, atlas_name):
    """
        subject_list : list of short subject IDs in string format
        atlas_name   : the atlas based on which the timeseries are generated e.g. aal, cc200
    returns:
        time_series  : list of timeseries arrays, each of shape (timepoints x regions)
    """

    timeseries = []
    for i in range(len(subject_list)):
        subject_folder = os.path.join(data_folder, subject_list[i])
        ro_file = [f for f in os.listdir(subject_folder) if f.endswith('_rois_' + atlas_name + '.1D')]
        fl = os.path.join(subject_folder, ro_file[0])
        print("Reading timeseries file %s" %fl)
        timeseries.append(np.loadtxt(fl, skiprows=0))

    return timeseries


# Compute connectivity matrices
def subject_connectivity(timeseries, subject, atlas_name, kind, save=True, save_path=data_folder):
    """
        timeseries   : timeseries table for subject (timepoints x regions)
        subject      : the subject ID
        atlas_name   : name of the parcellation atlas used
        kind         : the kind of connectivity to be used, e.g. lasso, partial correlation, correlation
        save         : save the connectivity matrix to a file
        save_path    : specify path to save the matrix if different from subject folder
    returns:
        connectivity : connectivity matrix (regions x regions)
    """

    print("Estimating %s matrix for subject %s" % (kind, subject))

    if kind in ['tangent', 'partial correlation', 'correlation']:
        conn_measure = connectome.ConnectivityMeasure(kind=kind)
        connectivity = conn_measure.fit_transform([timeseries])[0]

    if save:
        subject_file = os.path.join(save_path, subject,
                                    subject + '_' + atlas_name + '_' + kind.replace(' ', '_') + '.mat')
        sio.savemat(subject_file, {'connectivity': connectivity})

    return connectivity


# Get the list of subject IDs
def get_ids(num_subjects=None):
    """
    return:
        subject_IDs    : list of all subject IDs
    """

    subject_IDs = np.genfromtxt(os.path.join(data_folder, 'subject_IDs.txt'), dtype=str)

    if num_subjects is not None:
        subject_IDs = subject_IDs[:num_subjects]

    return subject_IDs


# Get phenotype values for a list of subjects
def get_subject_score(subject_list, score):
    scores_dict = {}

    with open(phenotype) as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            if row['SUB_ID'] in subject_list:
                scores_dict[row['SUB_ID']] = row[score]

    return scores_dict


# Dimensionality reduction step for the feature vector using a ridge classifier
def feature_selection(matrix, labels, train_ind, fnum):
    """
        matrix       : feature matrix (num_subjects x num_features)
        labels       : ground truth labels (num_subjects x 1)
        train_ind    : indices of the training samples
        fnum         : size of the feature vector after feature selection
    return:
        x_data      : feature matrix of lower dimension (num_subjects x fnum)
    """

    estimator = RidgeClassifier()
    selector = RFE(estimator, fnum, step=100, verbose=1)
    print(np.shape(train_ind))
    print(train_ind)
    print("at ferature selection")

    featureX = matrix[train_ind, :]
    featureY = labels[train_ind]
    selector = selector.fit(featureX, featureY.ravel())
    x_data = selector.transform(matrix)

    print("Number of labeled samples %d" % len(train_ind))
    print("Number of features selected %d" % x_data.shape[1])

    return x_data


# Make sure each site is represented in the training set when selecting a subset of the training set
def site_percentage(train_ind, perc, subject_list):
    """
        train_ind    : indices of the training samples
        perc         : percentage of training set used
        subject_list : list of subject IDs
    return:
        labeled_indices      : indices of the subset of training samples
    """

    train_list = subject_list[train_ind]
    sites = get_subject_score(train_list, score='SITE_ID')
    unique = np.unique(list(sites.values())).tolist()
    site = np.array([unique.index(sites[train_list[x]]) for x in range(len(train_list))])

    labeled_indices = []

    for i in np.unique(site):
        id_in_site = np.argwhere(site == i).flatten()

        num_nodes = len(id_in_site)
        labeled_num = int(round(perc * num_nodes))
        labeled_indices.extend(train_ind[id_in_site[:labeled_num]])

    return labeled_indices

def feature_selection(matrix, labels, train_ind, fnum):
    """
        matrix       : feature matrix (num_subjects x num_features)
        labels       : ground truth labels (num_subjects x 1)
        train_ind    : indices of the training samples
        fnum         : size of the feature vector after feature selection
    return:
        x_data      : feature matrix of lower dimension (num_subjects x fnum)
    """

    estimator = RidgeClassifier()
    selector = RFE(estimator, fnum, step=100, verbose=1)

    featureX = matrix[train_ind, :]
    featureY = labels[train_ind]
    selector = selector.fit(featureX, featureY.ravel())
    x_data = selector.transform(matrix)

    print("Number of labeled samples %d" % len(train_ind))
    print("Number of features selected %d" % x_data.shape[1])

    return x_data

# Load precomputed fMRI connectivity networks
def get_networks(subject_list, kind, atlas_name="aal", variable='connectivity'):
    """
        subject_list : list of subject IDs
        kind         : the kind of connectivity to be used, e.g. lasso, partial correlation, correlation
        atlas_name   : name of the parcellation atlas used
        variable     : variable name in the .mat file that has been used to save the precomputed networks
    return:
        matrix      : feature matrix of connectivity networks (num_subjects x network_size)
    """

    all_networks = []
    for subject in subject_list:
        fl = os.path.join(data_folder, subject,
                          subject + "_" + atlas_name + "_" + kind + ".mat")
        matrix = sio.loadmat(fl)[variable]
        all_networks.append(matrix)
    # all_networks=np.array(all_networks)

    idx = np.triu_indices_from(all_networks[0], 1)
    norm_networks = [np.arctanh(mat) for mat in all_networks]
    vec_networks = [mat[idx] for mat in norm_networks]
    matrix = np.vstack(vec_networks)

    return matrix

def create_affinity_graph_from_scores(scores, subject_list):
    """
        scores       : list of phenotypic information to be used to construct the affinity graph
        subject_list : list of subject IDs

    return:
        graph        : adjacency matrix of the population graph (num_subjects x num_subjects)
    """

    num_nodes = len(subject_list)
    graph = np.zeros((num_nodes, num_nodes))

    for l in scores:
        label_dict = get_subject_score(subject_list, l)

        # quantitative phenotypic scores
        if l in ['AGE_AT_SCAN', 'FIQ']:
            for k in range(num_nodes):
                for j in range(k + 1, num_nodes):
                    try:
                        val = abs(float(label_dict[subject_list[k]]) - float(label_dict[subject_list[j]]))
                        if val < 2:
                            graph[k, j] += 1
                            graph[j, k] += 1
                    except ValueError:  # missing label
                        pass

        else:
            for k in range(num_nodes):
                for j in range(k + 1, num_nodes):
                    if label_dict[subject_list[k]] == label_dict[subject_list[j]]:
                        graph[k, j] += 1
                        graph[j, k] += 1

    return graph

In [3]:
files = ['rois_ho']
num_subjects = 871
filemapping = {'func_preproc': 'func_preproc.nii.gz',
               'rois_ho': 'rois_ho.1D'}

if not os.path.exists(data_folder): os.makedirs(data_folder)
shutil.copyfile('./subject_IDs.txt', os.path.join(data_folder, 'subject_IDs.txt'))

'/data/zfzhu/zs/MMGL/miccai_ABIDE/data/ABIDE_pcp/cpac/filt_noglobal/subject_IDs.txt'

In [20]:
# Download database files
abide = datasets.fetch_abide_pcp(data_dir=root_folder, n_subjects=num_subjects, pipeline=pipeline,
                                 band_pass_filtering=True, global_signal_regression=False, derivatives=files)


#subject_IDs = Reader.get_ids(num_subjects)
#subject_IDs = subject_IDs.tolist()

 ...done. (1 seconds, 0 min)


 ...done. (2 seconds, 0 min)


 ...done. (2 seconds, 0 min)


 ...done. (1 seconds, 0 min)


 ...done. (2 seconds, 0 min)


...

 ...done. (2 seconds, 0 min)


 ...done. (2 seconds, 0 min)


 ...done. (2 seconds, 0 min)


 ...done. (1 seconds, 0 min)


 ...done. (1 seconds, 0 min)


In [4]:
subject_IDs = get_ids()
subject_IDs = subject_IDs.tolist()
labels = get_subject_score(subject_IDs, score='DX_GROUP')

for s, fname in zip(subject_IDs, fetch_filenames(subject_IDs, files[0])):
    subject_folder = os.path.join(data_folder, s)
    if not os.path.exists(subject_folder):
        os.mkdir(subject_folder)

    # Get the base filename for each subject
    base = fname.split(files[0])[0]

    # Move each subject file to the subject folder
    for fl in files:
        if not os.path.exists(os.path.join(subject_folder, base + filemapping[fl])):

            shutil.move(base + filemapping[fl], subject_folder)

time_series = get_timeseries(subject_IDs, 'ho')

# Compute and save connectivity matrices
for i in range(len(subject_IDs)):
        subject_connectivity(time_series[i], subject_IDs[i], 'ho', 'correlation')

Reading timeseries file /data/zfzhu/zs/MMGL/miccai_ABIDE/data/ABIDE_pcp/cpac/filt_noglobal/50003/Pitt_0050003_rois_ho.1D
Reading timeseries file /data/zfzhu/zs/MMGL/miccai_ABIDE/data/ABIDE_pcp/cpac/filt_noglobal/50004/Pitt_0050004_rois_ho.1D
Reading timeseries file /data/zfzhu/zs/MMGL/miccai_ABIDE/data/ABIDE_pcp/cpac/filt_noglobal/50005/Pitt_0050005_rois_ho.1D
Reading timeseries file /data/zfzhu/zs/MMGL/miccai_ABIDE/data/ABIDE_pcp/cpac/filt_noglobal/50006/Pitt_0050006_rois_ho.1D
Reading timeseries file /data/zfzhu/zs/MMGL/miccai_ABIDE/data/ABIDE_pcp/cpac/filt_noglobal/50007/Pitt_0050007_rois_ho.1D
Reading timeseries file /data/zfzhu/zs/MMGL/miccai_ABIDE/data/ABIDE_pcp/cpac/filt_noglobal/50008/Pitt_0050008_rois_ho.1D
Reading timeseries file /data/zfzhu/zs/MMGL/miccai_ABIDE/data/ABIDE_pcp/cpac/filt_noglobal/50010/Pitt_0050010_rois_ho.1D
Reading timeseries file /data/zfzhu/zs/MMGL/miccai_ABIDE/data/ABIDE_pcp/cpac/filt_noglobal/50011/Pitt_0050011_rois_ho.1D
Reading timeseries file /data/zf

In [5]:
SEX   = get_subject_score(subject_IDs, score='SEX')
AGE   = get_subject_score(subject_IDs, score='AGE_AT_SCAN')
FIQ   = get_subject_score(subject_IDs, score='FIQ')
LABEL = get_subject_score(subject_IDs, score='DX_GROUP')
EYE_S = get_subject_score(subject_IDs, score='EYE_STATUS_AT_SCAN')
BMI   = get_subject_score(subject_IDs, score='BMI')

In [6]:
ADS = []
Normal = []
for i in LABEL:
    if LABEL[i] == '1':
        ADS.append(i)
    elif LABEL[i] == '2':
        Normal.append(i)

In [7]:


# Get acquisition site
sites = get_subject_score(subject_IDs, score='SITE_ID')
unique = np.unique(list(sites.values())).tolist()

num_classes = 2
num_nodes = len(subject_IDs)

# Initialise variables for class labels and acquisition sites
y_data = np.zeros([num_nodes, num_classes])
y = np.zeros([num_nodes, 1])
site = np.zeros([num_nodes, 1], dtype=np.int)

# Get class labels and acquisition site for all subjects
for i in range(num_nodes):
    y_data[i, int(labels[subject_IDs[i]])-1] = 1
    y[i] = int(labels[subject_IDs[i]])
    site[i] = unique.index(sites[subject_IDs[i]])

# Compute feature vectors (vectorised connectivity networks)
features = get_networks(subject_IDs, kind='correlation', atlas_name='ho')

/home/zfzhu/anaconda3/envs/SS_env/lib/python3.6/site-packages/ipykernel_launcher.py:211: RuntimeWarning: divide by zero encountered in arctanh


# feature selection

In [8]:
label = y
label = label.reshape(-1)

In [9]:
from sklearn.model_selection import KFold,StratifiedKFold
skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

In [11]:
for train_index, test_index in skf.split(features, label):
    print(train_index)
    print(test_index)

[  0   1   5   6   7   8   9  11  12  13  15  16  17  18  19  21  22  23
  24  25  26  28  29  30  31  32  33  34  37  38  39  40  41  43  44  46
  47  48  50  51  52  54  55  56  58  59  60  62  64  65  66  68  70  71
  72  73  74  75  76  77  78  80  81  82  85  86  88  89  90  91  92  93
  94  95  98  99 100 101 102 103 104 105 106 107 108 109 110 112 113 115
 117 118 119 120 121 124 125 126 127 128 129 134 135 137 138 139 140 142
 143 144 149 150 151 153 155 156 157 158 159 160 161 163 165 166 167 168
 169 172 173 174 175 176 177 178 179 180 181 182 185 186 187 188 189 190
 191 193 194 195 196 198 199 200 201 203 205 206 207 208 209 211 212 213
 214 215 216 219 220 221 224 225 226 227 228 229 230 231 233 235 236 237
 238 239 240 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256
 257 258 259 260 261 262 263 264 265 267 268 269 272 273 274 276 277 279
 280 281 284 285 287 288 289 290 291 293 295 296 297 298 299 300 301 304
 305 306 308 309 310 311 312 313 314 315 317 318 31

In [12]:
feat_256_1 = feature_selection(features, label, train_index, 256)

Fitting estimator with 6105 features.
Fitting estimator with 6005 features.
Fitting estimator with 5905 features.
Fitting estimator with 5805 features.
Fitting estimator with 5705 features.
Fitting estimator with 5605 features.
Fitting estimator with 5505 features.
Fitting estimator with 5405 features.
Fitting estimator with 5305 features.
Fitting estimator with 5205 features.
Fitting estimator with 5105 features.
Fitting estimator with 5005 features.
Fitting estimator with 4905 features.
Fitting estimator with 4805 features.
Fitting estimator with 4705 features.
Fitting estimator with 4605 features.
Fitting estimator with 4505 features.
Fitting estimator with 4405 features.
Fitting estimator with 4305 features.
Fitting estimator with 4205 features.
Fitting estimator with 4105 features.
Fitting estimator with 4005 features.
Fitting estimator with 3905 features.
Fitting estimator with 3805 features.
Fitting estimator with 3705 features.
Fitting estimator with 3605 features.
Fitting esti

In [13]:
feat_256_1_pd = pd.DataFrame(feat_256_1)
feat_256_1_pd['label'] = label
feat_256_1_pd.to_csv("processed_standard_data_256_1.csv", index = False)

In [7]:
feat_dict = {}
for i in range(1):
    feat_dict[i] = list(range(i*256,(i+1)*256))

In [9]:
np.save('modal_feat_dict.npy',feat_dict)

,ST35TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST54TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST102TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST104SA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST110TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST121TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST32TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST23TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST51TS_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST93SA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,...,Ventricles,Hippocampus,WholeBrain,Fusiform,MidTemp,AV45,Entorhinal,FDG,ICV,LABEL
0,-0.045240,-1.295892,0.421234,-0.483017,0.025997,1.238236,-0.154253,-0.336338,0.849826,0.066977,...,-0.288704,0.430460,0.344339,0.402085,1.370027,-0.846159,0.830700,-0.140256,0.521384,1.0
1,-2.173109,-0.501574,-2.189485,-1.593304,0.044500,1.867784,1.934774,-0.771766,0.995551,-1.845264,...,1.026730,-1.190349,-1.528273,-1.728756,-1.095830,-1.216057,0.481848,1.286633,0.284877,1.0
2,-1.348017,-0.474184,-1.356618,0.168981,-0.251556,-1.504799,1.154694,-2.206534,-0.079174,-0.997639,...,-0.980602,1.189516,0.134473,-0.085066,-0.783282,-0.829864,0.966364,0.179889,0.020099,1.0
3,-1.279776,-0.426251,-0.796034,-1.215334,1.839342,1.478064,1.776114,-0.343477,2.015630,-1.906293,...,0.154781,-0.082127,-0.870265,-1.890088,-0.850180,-0.810352,0.094236,1.307737,-0.202742,1.0
4,-0.771073,-0.220824,0.469284,-1.512985,1.099201,-0.155766,-1.264875,-0.600450,0.503728,-1.540119,...,-0.878263,0.414386,0.195506,-0.709487,0.082070,-0.652844,0.214672,-1.109824,-0.619403,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,-0.343017,-0.248214,0.677501,1.510775,1.857846,-0.200733,-0.339357,1.169811,1.997414,0.467056,...,1.985485,0.376879,2.039405,0.324377,-0.117904,0.962720,-0.179862,-1.746598,1.812988,3.0
681,0.072630,1.347269,2.359253,-0.317655,1.709818,0.593697,0.586162,-0.057951,2.052061,-0.394130,...,-0.512797,-0.235724,-0.523839,-0.349350,-0.873918,-1.448125,-0.069116,-0.108871,-0.211957,3.0
682,-0.752462,2.196368,-0.395617,-0.936581,-0.677137,0.878493,0.136624,0.720108,-0.224900,0.589114,...,1.356993,-1.487721,-0.477471,-1.019528,-1.486066,-0.387314,-2.602442,-1.642633,0.503644,3.0
683,0.159482,-0.029092,1.670535,0.556401,2.209413,0.833526,1.115029,0.348924,2.070277,0.656924,...,0.828848,-1.489507,-0.298548,-1.107886,-0.158186,1.029188,-1.403611,-0.941263,-0.161559,3.0
